<img src="../media/onda_logo.jpg" alt="logo onda" height="200" width="200">

# AOI search via a Jupyter notebook
This trial notebook allows the geographical search of ONDA EO data offer by simutating a little 🌍 [ONDA Catalogue](https://catalogue.onda-dias.eu/catalogue/) via Jupyter Notebook.

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser("~"),"CLEOPE/Trials/modules"))
import aoi, buttons, qm
from ipywidgets import interact

Please note that every time you load the `button` module all the previous product lists placed in the *output* folder and named after _product_list*.txt_ will be erased, if present.

## Draw a polygon on the map
- Select an area of interest (AOI) choosing a shape among the buttons on the left side
- Only one polygon per time is supported by this version
- ⚠️ Do not scroll world window and draw your AOI in a world copy. This will generate non valid coordinates for the query (read more on this issue here 👉[Issue #96](https://github.com/jupyter-widgets/ipyleaflet/issues/96))

In [2]:
m,dc = aoi.define_map()
def handle_draw(self, action, geo_json):
    print("Action: %s"%action)
    print(geo_json["geometry"])
    buttons.save_aoi(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)
display(m)

Map(center=[40.853294, 14.305573], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

## Set filters
Remember to submit each field (even if not set).<br>👇

In [3]:
buttons._filters_()

DatePicker(value=None, description='Sensed from:')

DatePicker(value=None, description='Sensed to:')

Button(description='Submit sensing range', style=ButtonStyle())

Select(options={'All': ['-'], 'Sentinel-1': ['All S1', 'SLC', 'GRD', 'OCN', 'RAW'], 'Sentinel-2': ['All S2', '…

Button(description='Submit mission', style=ButtonStyle())

Output()

## Send the query

Running the cell below you will get results from the ONDA catalogue using the [OData API](https://www.onda-dias.eu/cms/knowledge-base/odata-odata-open-data-protocol/) provided interface withing your own CLEOPE workspace. Wait untill the process is concluded; its overall duration will depend on the query.

In [5]:
polysel = buttons.read_aoi()
item = buttons.read_mp()
sensing = buttons.read_sen()
n = aoi.count(polysel,item,sensing)
df = aoi.search(n,polysel,item,sensing)

Searching for products...
https://catalogue.onda-dias.eu/dias-catalogue/Products/$count?$search=%22(%20(%20(%20platformName:Sentinel-5 Precursor%20AND%20productType:L2__CO____%20)%20)%20AND%20(%20(%20beginPosition:[2020-07-01T00:00:00.000Z%20TO%202020-07-02T23:59:59.999Z]%20AND%20endPosition:[2020-07-01T00:00:00.000Z%20TO%202020-07-02T23:59:59.999Z]%20)%20)%20)%20AND%20footprint:%22Intersects(POLYGON((19.682717%2032.235262,19.682717%2041.099362,33.228846%2041.099362,33.228846%2032.235262,19.682717%2032.235262)))%22%22
2 products found


## Visualise layers on the map

Results are here interactively visualised so that you can choose products depending on the their footprint intersection upon the drawn AOI.

In [6]:
@interact
def _iFP_(product=[f for f in df.iloc[:,1]]): 
    keys = df.index[df['name'] == product].tolist()
    m = aoi.update_Map(keys,polysel,df)
    display(m)
    display(df.loc[keys])
    aoi.warning(df.loc[keys].iloc[:,6].any())
    print("Find item using ENS at pseudopath:\n%s"%os.path.join(str(df.loc[keys].iloc[:,2].any()),str(df.loc[keys].iloc[:,1].any())))

interactive(children=(Dropdown(description='product', options=('S5P_OFFL_L2__CO_____20200701T092050_20200701T1…

## Save up products in a text file

Choose products you wish to collect and submit each of them. This will produce a products list in the *output* folder of CLEOPE workspace. 

In [7]:
sel = buttons.select_product(df)
sel.display_widgets() 

## Work with the product list text file
Product list file named *product_list.txt* contains the name of all the products you decided to save up previously. Now in order to work with this product list you can choose to proceed with one of the following options.

### 1. Use ONDA Advanced API (ENS)
In this case you need to convert your product list into ENS format. Run the following cell to obtain a product list compliant to ENS [ENS path to products](../readme.md/#pseudopaths) and then use this list to start your own EO data processing.<br>The input file list can be put as the `file` argument of the `read_product_list` function, while the output filename can be set as `outfile` argument of the `pseudopath` function.

In [ ]:
pseudopaths = qm.pseudopath(dataframe=qm.read_product_list(file="outputs/product_list.txt"),
                            outfile="outputs/product_list_remote.txt")

### 2. Download your custom product list 

In this case you need to download products using the [OData API](https://www.onda-dias.eu/cms/knowledge-base/odata-odata-open-data-protocol/) provided interface. Just provide your own ONDA credentials and the input file list of products (*product_list.txt* file in the example below). The downloaded files will be put in the *local_files* folder in your own workspace and the *list_local.txt* file will be created containing the path to downloaded products.

In [ ]:
username = "username"; password = "password" # ONDA auth required
r = qm.download_list(file='outputs/product_list.txt',username=username,password=password) # download products from your custom list